In [2]:
import os
import shutil
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB4
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau


In [3]:
def split_dataset(root_dir, output_dir, test_size=0.15, val_size=0.15, seed=42):
    """
    Splits dataset into train/val/test sets and creates directory structure
    """
    # Create output directories
    os.makedirs(os.path.join(output_dir, 'train'), exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'val'), exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'test'), exist_ok=True)

    # Process each class
    for class_name in ['Healthy', 'Unhealthy']:
        # Create class directories in train/val/test
        os.makedirs(os.path.join(output_dir, 'train', class_name), exist_ok=True)
        os.makedirs(os.path.join(output_dir, 'val', class_name), exist_ok=True)
        os.makedirs(os.path.join(output_dir, 'test', class_name), exist_ok=True)

        # Get list of images
        class_dir = os.path.join(root_dir, class_name)
        images = [f for f in os.listdir(class_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]
        
        # Split into temp (85%) and test (15%)
        temp_images, test_images = train_test_split(images, 
                                                  test_size=test_size, 
                                                  random_state=seed)
        
        # Split temp into train (82.35%) and val (17.65%) to get final 70-15-15 split
        train_images, val_images = train_test_split(temp_images, 
                                                   test_size=val_size/(1-test_size), 
                                                   random_state=seed)
        
        # Function to copy images
        def copy_files(file_list, split_name):
            for f in file_list:
                src = os.path.join(class_dir, f)
                dst = os.path.join(output_dir, split_name, class_name, f)
                shutil.copyfile(src, dst)
        
        # Copy files to respective directories
        copy_files(train_images, 'train')
        copy_files(val_images, 'val')
        copy_files(test_images, 'test')

# Usage - modify these paths according to your setup
input_dir = './Hibiscus'  # Should contain 'diseased' and 'not_diseased' folders
output_dir = 'images_hibiscus'  # New directory that will be created

split_dataset(input_dir, output_dir)



In [5]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG19
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix

# Define paths
base_dir = './images_hibiscus'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'val')
test_dir = os.path.join(base_dir, 'test')

# Image preprocessing and augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255  # Normalize pixel values to [0, 1]
)

validation_datagen = ImageDataGenerator(rescale=1./255)  # Only rescale for validation
test_datagen = ImageDataGenerator(rescale=1./255)  # Only rescale for testing

# Create data generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),  # Resize images to 224x224 (VGG19 input size)
    batch_size=32,
    class_mode='binary'  # Binary classification (healthy/unhealthy)
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False  # Do not shuffle for evaluation
)

# Load the VGG19 model (pre-trained on ImageNet)
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model (do not train the pre-trained layers)
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers on top of the base model
x = base_model.output
x = Flatten()(x)  # Flatten the output of the base model
x = Dense(512, activation='relu')(x)  # Add a fully connected layer
x = Dropout(0.5)(x)  # Add dropout for regularization
predictions = Dense(1, activation='sigmoid')(x)  # Output layer for binary classification

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10  # Number of epochs
)


# Save the trained model
model_save_path = './saved_model/hibiscus_health_classifier.h5'  # Path to save the model
os.makedirs(os.path.dirname(model_save_path), exist_ok=True)  # Ensure the directory exists
model.save(model_save_path)  # Save the model
print(f'Model saved to {model_save_path}')

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
print(f'Test Accuracy: {test_acc:.4f}')

# Predictions on the test set
y_pred = model.predict(test_generator)
y_pred = np.round(y_pred).astype(int)  # Convert probabilities to binary predictions

# Classification report and confusion matrix
print('Classification Report:')
print(classification_report(test_generator.classes, y_pred))

print('Confusion Matrix:')
print(confusion_matrix(test_generator.classes, y_pred))

Found 2092 images belonging to 2 classes.
Found 450 images belonging to 2 classes.
Found 450 images belonging to 2 classes.
Epoch 1/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 334s 5s/step - accuracy: 0.6623 - loss: 0.6640 - val_accuracy: 0.9621 - val_loss: 0.1897
Epoch 2/10
 1/65 ━━━━━━━━━━━━━━━━━━━━ 4:40 4s/step - accuracy: 0.8750 - loss: 0.2908

C:\Users\kavit\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


65/65 ━━━━━━━━━━━━━━━━━━━━ 63s 923ms/step - accuracy: 0.8750 - loss: 0.2908 - val_accuracy: 0.9710 - val_loss: 0.1798
Epoch 3/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 338s 5s/step - accuracy: 0.9167 - loss: 0.2135 - val_accuracy: 0.9777 - val_loss: 0.1295
Epoch 4/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 63s 919ms/step - accuracy: 0.9062 - loss: 0.2537 - val_accuracy: 0.9777 - val_loss: 0.1259
Epoch 5/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 787s 12s/step - accuracy: 0.9533 - loss: 0.1452 - val_accuracy: 0.9821 - val_loss: 0.1006
Epoch 6/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 211s 3s/step - accuracy: 0.9375 - loss: 0.0830 - val_accuracy: 0.9844 - val_loss: 0.0989
Epoch 7/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 1418s 22s/step - accuracy: 0.9588 - loss: 0.1246 - val_accuracy: 0.9844 - val_loss: 0.0865
Epoch 8/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 287s 4s/step - accuracy: 1.0000 - loss: 0.0730 - val_accuracy: 0.9844 - val_loss: 0.0874
Epoch 9/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 1605s 25s/step - accuracy: 0.9737 - loss: 0.0912 - val_accuracy: 0.9911 - val

Model saved to ./saved_model/hibiscus_health_classifier.h5
14/14 ━━━━━━━━━━━━━━━━━━━━ 121s 9s/step - accuracy: 0.9923 - loss: 0.1059
Test Accuracy: 0.9888
15/15 ━━━━━━━━━━━━━━━━━━━━ 128s 8s/step
Classification Report:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       219
           1       1.00      0.98      0.99       231

    accuracy                           0.99       450
   macro avg       0.99      0.99      0.99       450
weighted avg       0.99      0.99      0.99       450

Confusion Matrix:
[[218   1]
 [  4 227]]
